In [1]:
import pandas  as pd
train_df = pd.read_csv('fashion-mnist_train.csv')
test_df = pd.read_csv('fashion-mnist_test.csv')
print(train_df.shape)
print(test_df.shape)

(60000, 785)
(10000, 785)


loading the dataset
first, i imported the pandas library , then, loadeded the fashion-mnist training and test datasets 

In [2]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image




class  FashionMNISTCSV(Dataset):
    def __init__(self, dataframe, transform=None):
        self.labels = dataframe.iloc[:, 0].values.astype(np.int64)
        self.images = dataframe.iloc[:, 1:].values.astype(np.uint8)
        self.transform = transform
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        image = self.images[idx].reshape(28, 28) 
        image = Image.fromarray(image)  
        image = image.convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label
transform = transforms.Compose([
    transforms.Resize((224, 224)),                    
    transforms.ToTensor(),                            
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
])








train_dataset = FashionMNISTCSV(train_df, transform=transform)
test_dataset = FashionMNISTCSV(test_df, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


creating a custom dataset and dataloader
defined a custom dataset class to load images and labels from the csv dataframes , the images are reshaped to 28x28 and converted to rgb since pretrained models expect 3 channels ,also applied transforms to resize images to 224x224, convert them to tensors, and normalize them .
also created dataset objects for training and testing, and wrapped them in dataloaders to load data in batches of 64

In [3]:
import torch
import torch.nn as nn
import torchvision.models as models
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)
print(model.fc)

C:\Users\LENOVO\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\LENOVO\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Linear(in_features=2048, out_features=10, bias=True)


loading pretrained resnet50 and modifying final layer
i loaded the pretrained resnet50 model and froze all its parameters so they won’t update during training , then, i replaced the final fully connected layer to output 10 classes, matching the number of categories in the fashion mnist dataset

In [4]:
from torchvision.models import resnet50, ResNet50_Weights
import torch.nn as nn
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
for param in model.parameters():
    param.requires_grad = False
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)
print(model.fc)

Linear(in_features=2048, out_features=10, bias=True)


loading resnet50 with updated weights syntax
i loaded the resnet50 model using the recommended weights syntax , all layers were frozen to prevent training them , then i replaced the final fully connected layer to output 10 classes for fashion mnist,then, i printed the modified layer to confirm the update

In [5]:
import torch
import torch.optim as optim
import torch.nn as nn
device = torch.device('cuda')
print(f"Using device: {device}")
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

Using device: cuda


setting device, loss function, and optimizer
i set the device to use gpu , then i moved the model to that device , i used cross entropy loss since it's a multi-class classification task and chose the adam optimizer to update only the final layer's parameters with a learning rate of 0.001

In [6]:
model.train()  
for batch_idx, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()     
    outputs = model(images)   
    loss = criterion(outputs, labels) 
    loss.backward()
    optimizer.step()            
    if batch_idx % 100 == 0:   
        print(f'Batch {batch_idx}, Loss: {loss.item():.4f}')

Batch 0, Loss: 2.3451
Batch 100, Loss: 1.0641
Batch 200, Loss: 0.6754
Batch 300, Loss: 0.6531
Batch 400, Loss: 0.5825
Batch 500, Loss: 0.5588
Batch 600, Loss: 0.4380
Batch 700, Loss: 0.6069
Batch 800, Loss: 0.4584
Batch 900, Loss: 0.4279


training loop for one epoch
i set the model to training mode and looped over the training data , each batch was moved to the selected device, loss was calculated, and backpropagation was done to update the model’s final layer , i printed the loss every 100 batches to monitor training progress

In [7]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = 100 * correct / total
print(f'Validation Accuracy: {accuracy:.2f}%')

Validation Accuracy: 83.46%


evaluating model on test data
i set the model to evaluation mode and disabled gradient calculation , then i looped through the test data, made predictions, and compared them with the true labels to count correct predictions. finally, i calculated and printed the overall validation accuracy

In [8]:
num_epochs = 5 
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if batch_idx % 100 == 99:  
            print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}], Loss: {running_loss/100:.4f}')
            running_loss = 0.0
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Validation Accuracy after epoch {epoch+1}: {accuracy:.2f}%')

Epoch [1/5], Batch [100], Loss: 0.4920
Epoch [1/5], Batch [200], Loss: 0.4823
Epoch [1/5], Batch [300], Loss: 0.4914
Epoch [1/5], Batch [400], Loss: 0.4764
Epoch [1/5], Batch [500], Loss: 0.4739
Epoch [1/5], Batch [600], Loss: 0.4790
Epoch [1/5], Batch [700], Loss: 0.4491
Epoch [1/5], Batch [800], Loss: 0.4611
Epoch [1/5], Batch [900], Loss: 0.4570
Validation Accuracy after epoch 1: 84.89%
Epoch [2/5], Batch [100], Loss: 0.4250
Epoch [2/5], Batch [200], Loss: 0.4353
Epoch [2/5], Batch [300], Loss: 0.4470
Epoch [2/5], Batch [400], Loss: 0.4265
Epoch [2/5], Batch [500], Loss: 0.4250
Epoch [2/5], Batch [600], Loss: 0.4210
Epoch [2/5], Batch [700], Loss: 0.4120
Epoch [2/5], Batch [800], Loss: 0.4177
Epoch [2/5], Batch [900], Loss: 0.4098
Validation Accuracy after epoch 2: 85.05%
Epoch [3/5], Batch [100], Loss: 0.4024
Epoch [3/5], Batch [200], Loss: 0.4026
Epoch [3/5], Batch [300], Loss: 0.3938
Epoch [3/5], Batch [400], Loss: 0.3943
Epoch [3/5], Batch [500], Loss: 0.4100
Epoch [3/5], Batch 

training and validating for multiple epochs
i trained the model for 5 epochs , in each epoch, i ran the training loop and printed the average loss every 100 batches , after each epoch, i evaluated the model on the test set and printed the validation accuracy to track progress

In [9]:
for param in model.layer4.parameters():
    param.requires_grad = True
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

fine-tuning last resnet block
i unfroze the parameters of layer4 to fine-tune the last block of resnet50 , then, i created a new optimizer that only updates the trainable parameters and added a learning rate scheduler to gradually reduce the learning rate after every epoch

In [10]:
num_finetune_epochs = 3  
for epoch in range(num_finetune_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if batch_idx % 100 == 0:
            print(f"Fine-Tune Epoch [{epoch+1}/{num_finetune_epochs}], Batch [{batch_idx}], Loss: {loss.item():.4f}")
    scheduler.step()
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Validation Accuracy after fine-tune epoch {epoch+1}: {accuracy:.2f}%\n")

Fine-Tune Epoch [1/3], Batch [0], Loss: 0.5315
Fine-Tune Epoch [1/3], Batch [100], Loss: 0.4464
Fine-Tune Epoch [1/3], Batch [200], Loss: 0.3593
Fine-Tune Epoch [1/3], Batch [300], Loss: 0.3219
Fine-Tune Epoch [1/3], Batch [400], Loss: 0.2225
Fine-Tune Epoch [1/3], Batch [500], Loss: 0.3468
Fine-Tune Epoch [1/3], Batch [600], Loss: 0.4317
Fine-Tune Epoch [1/3], Batch [700], Loss: 0.1693
Fine-Tune Epoch [1/3], Batch [800], Loss: 0.4337
Fine-Tune Epoch [1/3], Batch [900], Loss: 0.2862
Validation Accuracy after fine-tune epoch 1: 88.38%

Fine-Tune Epoch [2/3], Batch [0], Loss: 0.1925
Fine-Tune Epoch [2/3], Batch [100], Loss: 0.1518
Fine-Tune Epoch [2/3], Batch [200], Loss: 0.2908
Fine-Tune Epoch [2/3], Batch [300], Loss: 0.2800
Fine-Tune Epoch [2/3], Batch [400], Loss: 0.3049
Fine-Tune Epoch [2/3], Batch [500], Loss: 0.2674
Fine-Tune Epoch [2/3], Batch [600], Loss: 0.2793
Fine-Tune Epoch [2/3], Batch [700], Loss: 0.2903
Fine-Tune Epoch [2/3], Batch [800], Loss: 0.2903
Fine-Tune Epoch [2/3

fine-tuning and validating model
i fine-tuned the model for 3 more epochs with layer 4 unfrozen , after training on each epoch, i used the scheduler to adjust the learning rate , then, i evaluated the model on the test set and printed the validation accuracy to track improvement